### Import rdflib and create functions

* g is the ontology graph where the infered relations are added to
* Serialize is a function that transforms the ontology into a turle format
* Save is a function that can save the ontology as a turtle file
* Load is a function that loads the ontology into this file
* Shorten is a function that shows the prefix + subject, property or object  vb. kinship1:has_mother
* Relation shows only the subject, property or object

In [1]:
import rdflib
from rdflib import Graph, RDF, RDFS, Namespace, Literal, URIRef 
import pandas as pd

pd.set_option('display.max_rows', 999)


df = pd.DataFrame()
g = Graph()

KIN = Namespace('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#')
g.bind('kinship1',KIN)

def serialize():
    print(g.serialize(format='turtle').decode("utf-8"))

def save(filename):
    with open(filename, 'wb') as f:
        g.serialize(f, format='turtle')
        
def load(filename):
    with open(filename, 'r') as f:
        g.load(f, format='turtle') 
    
def shorten(uri):
    if isinstance(uri, Literal):
        return uri    
    components = g.namespace_manager.compute_qname(uri)
    return '%s:%s'%(components[0], components[2])

def relation(uri):
    if isinstance(uri, Literal):
        return uri    
    components = g.namespace_manager.compute_qname(uri)
    return '%s'%(components[2]) 

## Create counter that counts the amount of newly created relations in this ontology.

In [2]:
new_relations_counter = 0

### Load in the ontology and print in turtle format

In [3]:
load('kinship.ttl')
serialize()

@prefix : <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1> .
@prefix kinship1: <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

kinship1:Andrea_Waltham a kinship1:Human,
        kinship1:woman,
        owl:NamedIndividual ;
    kinship1:has_Daughter kinship1:Emily_Waltham .

kinship1:Brother_in_law a owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Brother in law"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    rdfs:subPropertyOf kinship1:In_law ;
    owl:propertyDisjointWith kinship1:Sister_in_law .

kinship1:Cousins_husband a owl:AsymmetricProperty,
        owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Cousins husband"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    owl:propertyDisjointWith kinship1:Cousins_wife .

kinship1:Daugther_in_law

### Create empty lists that have to be filled later.

In [4]:
parent  = []
parent2 = []
father  = []
mother  = []
child   = []
child2  = []
son     = []
daughter = []
grandchild = []
grandchild2 = []
grandparent = []
grandparent2 = []
couple  = []
couple2 = []
marriage = []
marriage2 = []
marriage3 = []
marriage4 = []
marriage5 = []
marriage6 = []
marriage7 = []
marriage8 = []
cousin1 = []
cousin2 = []
sibling1 = []
sibling2 = []
sister = []
sister2 = []
brother = []
brother2 = []

### Create a dataframe that later shows the inferred gender relations and dataframe that shows the inferred relations.

In [5]:
df = pd.DataFrame(columns = ["Relation", "Subject", "Property", "Object"])
df2 = pd.DataFrame(columns = ["Subject", "Property", "Object"])
df5 = pd.DataFrame(columns = ["Subject", "Property", "Object"])

 ### Fill the lists with the right values and add gender to individuals
 The known relations will be added seperatly in lists and the genders of the individuals will be added to the ontology by determining what their relation is with other individuals. For example, if someone is a mother she will automatically also be a woman.

In [6]:
for s,p,o in g:
# PARENT RELATIONS
    if shorten(p) == "kinship1:has_parent" or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        parent.append(o)
        child.append(s)    
    if shorten(p) == "kinship1:has_Father":
        df = df.append({'Relation': "Father", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Mother":
        df = df.append({'Relation': "Mother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# GRANDPARENT RELATIONS
    if shorten(p) == "kinship1:has_Grandparent" or shorten(p) == "kinship1:has_Grandfather " or shorten(p) == "kinship1:has_Grandmother":
        grandparent.append(o)
        grandchild.append(s)    
    if shorten(p) == "kinship1:has_Grandfather":
        df = df.append({'Relation': "Grandfather", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Grandmother":
        df = df.append({'Relation': "Grandmother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# CHILD RELATIONS       
    if shorten(p) == "kinship1:has_Child" or shorten(p) == "kinship1:has_Son" or shorten(p) == "kinship1:has_Dauther": #typo in ontology daughter
        child2.append(o)
        parent2.append(s)    
    if shorten(p) == "kinship1:has_Son":
        df = df.append({'Relation': "Son", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Dauther":
        df = df.append({'Relation': "Daughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# GRANDCHILD RELATIONS       
    if shorten(p) == "kinship1:has_Grandchild" or shorten(p) == "kinship1:has_Grandson" or shorten(p) == "kinship1:has_Granddaughter": 
        grandchild2.append(o)
        grandparent2.append(s)   
    if shorten(p) == "kinship1:has_Grandson":
        df = df.append({'Relation': "Grandson", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Granddaughter":
        df = df.append({'Relation': "Granddaughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# SIBLING RELATIONS
    if shorten(p) == "kinship1:has_Sibling" or  shorten(p) == "kinship1:has_Sister" or shorten(p) == "kinship1:has_Brother": 
        sibling1.append(o)
        sibling2.append(s)
    if shorten(p) == "kinship1:has_Brother":
        brother.append(o)
        brother2.append(s)
        df = df.append({'Relation': "Brother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Sister":
        sister.append(o)
        sister2.append(s)
        df = df.append({'Relation': "Sister", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# COUSIN RELATIONS
    if shorten(p) == "kinship1:has_Cousin":
        cousin1.append(o)
        cousin2.append(s)

#RELATIONSHIPS 
    if shorten(p) == "kinship1:has_Husband" or shorten(p) == "kinship1:has_Wife" or shorten(p) == "kinship1:has_Partner":
        couple.append(o)
        couple2.append(s)
    if shorten(p) == "kinship1:has_Husband":
        marriage.append(o)
        marriage2.append(s)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
        df = df.append({'Relation': "Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Wife":
        marriage3.append(o)
        marriage4.append(s)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        df = df.append({'Relation': "Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_husband":
        marriage5.append(o)
        marriage6.append(s)
        g.add((o, RDFS.domain, KIN.man)) 
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_wife":
        marriage7.append(o)
        marriage8.append(s)
        g.add((o, RDFS.domain, KIN.woman))  
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)

### Print a table with the inferred gender relations that are added to the ontology

In [7]:
df = df.drop(columns=["Property"])
df = df.sort_values(by=['Relation'])
df.reset_index(drop=True)

,Relation,Subject,Object
0,Brother,Frank_Buffay,man
1,Ex_Husband,Duncan,man
2,Ex_Wife,Carol_Willick,woman
3,Ex_Wife,Emily_Waltham,woman
4,Father,Leonard_Green,man
5,Father,Jack_Geller,man
6,Father,Charles_Bing,man
7,Father,Jack_Geller,man
8,Father,Ross,man
9,Grandmother,Frances,woman


In [8]:
print (g.serialize(format='turtle').decode("utf-8")) 

@prefix : <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1> .
@prefix kinship1: <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

kinship1:Andrea_Waltham a kinship1:Human,
        kinship1:woman,
        owl:NamedIndividual ;
    kinship1:has_Daughter kinship1:Emily_Waltham .

kinship1:Brother_in_law a owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Brother in law"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    rdfs:subPropertyOf kinship1:In_law ;
    owl:propertyDisjointWith kinship1:Sister_in_law .

kinship1:Cousins_husband a owl:AsymmetricProperty,
        owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Cousins husband"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    owl:propertyDisjointWith kinship1:Cousins_wife .

kinship1:Daugther_in_law

### Extract all the necessary properties and print the ones that exist in the ontology to show what they look like.
The ontology might consist of many more relations,but these are similar to these ones. They will however not be detected if they are not equal to the default properties. That would have been a lot more work to find all the relations and due to limited time, only the default properties can be used. 

In [9]:
child_property = None
parent_property = None 
grandchild_property= None
grandparent_property= None
husband_property= None
wife_property= None
ex_husband_property= None
ex_wife_property= None
cousin_property= None
partner_property= None
father_property= None
mother_property= None
son_property= None
daughter_property= None
grandson_property= None
granddaugther_property= None
grandfather_property= None
grandmother_property= None
brother_property= None
sister_property= None
twin_property= None
sibling_property= None

In [10]:
propery_list = []

for s,p,o in g:
    if shorten(p) == "kinship1:has_Child":
        child_property = p
        propery_list.append(p)
        print("child_property", child_property)
    if shorten(p) == "kinship1:has_parent":
        parent_property = p
        propery_list.append(p)
        print("parent_property", parent_property)
    if shorten(p) == "kinship1:has_Grandchild":
        grandchild_property = p
        propery_list.append(p)
        print("grandchild_property", grandchild_property)
    if shorten(p) == "kinship1:has_Grandparent":
        grandparent_property = p
        propery_list.append(p)
        print("grandparent_property", grandparent_property)

    if shorten(p) == "kinship1:has_Husband":
        husband_property = p
        propery_list.append(p)
        print("husband_property", husband_property)
    if shorten(p) == "kinship1:has_Wife":
        wife_property = p
        propery_list.append(p)
        print("wife_property", wife_property)
    if shorten(p) == "kinship1:has_Ex_husband":
        ex_husband_property = p
        propery_list.append(p)
        print("ex_husband_property", ex_husband_property)
    if shorten(p) == "kinship1:has_Ex_wife":
        ex_wife_property = p
        propery_list.append(p)
        print("ex_wife_property", ex_wife_property)

    if shorten(p) ==  "kinship1:has_Cousin":
        cousin_property = p
        propery_list.append(p)
        print("cousin_property", cousin_property)
    if shorten(p) == "kinship1:has_Sibling":
        sibling_property = p
        propery_list.append(p)
        print("sibling_property", sibling_property)

    if shorten(p) == "kinship1:has_Partner":
        partner_property = p
        propery_list.append(p)
        print("partner_property", partner_property)
    if shorten(p) == "kinship1:has_Father":
        father_property = p
        propery_list.append(p)
        print("father_property", father_property)
    if shorten(p) == "kinship1:has_Mother":
        mother_property = p
        propery_list.append(p)
        print("mother_property", mother_property)
    if shorten(p) ==  "kinship1:has_Son"  :
        son_property = p
        propery_list.append(p)
        print("son_property", son_property)
    if shorten(p) == "kinship1:has_Dauther":
        daugther_property = p
        propery_list.append(p)
        print("daugther_property", daugther_property)
    if shorten(p) == "kinship1:has_Grandson":
        grandson_property = p
        propery_list.append(p)
        print("grandson_property", grandson_property)
    if shorten(p) == "kinship1:has_Granddaughter":
        granddaugther_property = p
        propery_list.append(p)
        print("granddaugther_property", granddaugther_property)
    if shorten(p) == "kinship1:has_Grandfather":
        grandfather_property = p
        propery_list.append(p)
        print("grandfather_property", grandfather_property)
    if shorten(p) == "kinship1:has_Grandmother":
        grandmother_property = p
        propery_list.append(p)
        print("grandmother_property", grandmother_property)
    if shorten(p) == "kinship1:has_Brother":
        brother_property = p
        propery_list.append(p)
        print("brother_property", brother_property)
    if shorten(p) == "kinship1:has_Sister":
        sister_property = p
        propery_list.append(p)
        print("sister_property", sister_property)
    if shorten(p) == "kinship1:has_twin":
        twin_property = p
        propery_list.append(p)
        print("twin_property", twin_property)


cousin_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Cousin
father_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Father
wife_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Wife
son_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Son
sister_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Sister
sister_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Sister
sister_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Sister
husband_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Husband
sister_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Sister
child_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Child
cousin_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Cousin
sister_property http://www.semanticweb.org

### Divine some necessary properties
These property names are used as a default for these relations. And since these are very important kinship relations, they are added if they are missing in the input data.

In [11]:
if sibling_property == None:
    sibling_property =  rdflib.term.URIRef(KIN.has_Sibling)
SymmetricProperty = rdflib.term.URIRef(KIN.SymmetricProperty)
SymmetricProperty = URIRef(SymmetricProperty)
sibling_property=URIRef(sibling_property)
print(sibling_property, RDF.type, SymmetricProperty)
g.add((sibling_property, RDF.type, SymmetricProperty))

if parent_property == None:
    parent_property =   rdflib.term.URIRef(KIN.has_parent)
IrreflexiveProperty = rdflib.term.URIRef(KIN.IrreflexiveProperty)
IrreflexiveProperty = URIRef(IrreflexiveProperty)
parent_property=URIRef(parent_property)
print(parent_property, RDF.type, IrreflexiveProperty)
g.add((parent_property, RDF.type, IrreflexiveProperty))

if child_property == None:
    child_property =   rdflib.term.URIRef(KIN.has_Child)
IrreflexiveProperty = rdflib.term.URIRef(KIN.IrreflexiveProperty)
IrreflexiveProperty = URIRef(IrreflexiveProperty)
child_property=URIRef(child_property)
print(child_property, RDF.type, IrreflexiveProperty)
g.add((child_property, RDF.type, IrreflexiveProperty))

if daughter_property == None:
    daughter_property =   rdflib.term.URIRef(KIN.has_daughter)
IrreflexiveProperty = rdflib.term.URIRef(KIN.IrreflexiveProperty)
IrreflexiveProperty = URIRef(IrreflexiveProperty)
daughter_property=URIRef(daughter_property)
print(daughter_property, RDF.type, IrreflexiveProperty)
g.add((daughter_property, RDF.type, IrreflexiveProperty))

http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Sibling http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#SymmetricProperty
http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_parent http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#IrreflexiveProperty
http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Child http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#IrreflexiveProperty
http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_daughter http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#IrreflexiveProperty


In [12]:
property_list = list(dict.fromkeys(propery_list))
property_list

[rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Cousin'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Father'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Wife'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Son'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Sister'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Husband'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Child'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_parent'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_twin'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Sibling'),
 rdflib.term.URIRef('http://www.semanticweb.or

### Create functions to iterate through the lists and add the reversed relations with the right property to the ontology.
All the added relations are also added to a dataframe.

In [13]:
def create_inferrence(new_subject, new_property, new_object, ignore_index=True):
    global df2
    global new_relations_counter
    for i in range(len(new_subject)):
        g.add((new_subject[i], new_property, new_object[i]))
        new_relations_counter +=1
        data = [[ relation(new_subject[i]), relation(new_property), relation(new_object[i])]]
        df3 = pd.DataFrame(data, columns = ['Subject', 'Property', "Object"])
        df2 = df2.append(df3)
        
def create_gender_based_inferrence(subject_list, new_property, relationship, ignore_index=True):
    global df5       
    global new_relations_counter
    for x in subject_list:
        if x[1] in relationship:
            g.add((x[0], new_property, x[1]))
            new_relations_counter +=1
            data = [[ relation(x[0]), relation(new_property), relation(x[1])]]
            df3 = pd.DataFrame(data, columns = ['Subject', 'Property', "Object"])
            df5 = df5.append(df3)
            
def make_word_classes(synom):
    # List of classes
    word_classes = []
    for w1, w2 in synom:
        # Go through existing classes
        for wcls in word_classes:
            # If one of the words is already in the current class
            if w1 in wcls or w2 in wcls:
                # Add both words and continue to next pair of words
                wcls.add(w1)
                wcls.add(w2)
                break
        else:  # Note this else goes with the for loop, not the if block
            # If there was no matching class, add a new one
            word_classes.append({w1, w2})
    return word_classes

In [14]:
woman_list = []
man_list = []
has_parent_relation = []
has_child_relation = []
has_grandparent_relation = []
has_grandchild_relation = []
has_sibling_relation = []
has_twin_relation=[]

for s,p,o in g:
    if shorten(o) ==  "kinship1:woman":
        woman_list.append(s)
    if shorten(o) ==  "kinship1:man":
        man_list.append(s)
    if shorten(p) == "kinship1:has_parent" or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        has_parent_relation.append((o, s))
    if shorten(p) == "kinship1:has_Child" or shorten(p) == "kinship1:has_Son" or shorten(p) == "kinship1:has_Dauther":
        has_child_relation.append((o, s))
    if shorten(p) == "kinship1:has_Grandparent" or shorten(p) == "kinship1:has_Grandfather " or shorten(p) == "kinship1:has_Grandmother":
        has_grandparent_relation.append((o, s))
    if shorten(p) == "kinship1:has_Grandchild" or shorten(p) == "kinship1:has_Grandson" or shorten(p) == "kinship1:has_Granddaughter":
        has_grandchild_relation.append((o, s))
    if shorten(p) == "kinship1:has_Sibling" or shorten(p) == "kinship1:has_Sister" or shorten(p) == "kinship1:has_Brother":
        has_sibling_relation.append((o,s))
    if shorten(p) == "kinship1:has_twin":
        has_twin_relation.append((o,s))

In [15]:
mothers = set(woman_list) & set([x[0] for x in has_parent_relation])
fathers = set(man_list) & set([x[0] for x in has_parent_relation])
daughters = set(woman_list) & set([x[0] for x in has_child_relation])
sons = set(man_list) & set([x[0] for x in has_child_relation])

daughters2 =  set(woman_list) & set([x[1] for x in has_parent_relation])
sons2 = set(man_list) & set([x[1] for x in has_parent_relation])
sons.update(sons2)
daughters.update(daughters2)                                    
brothers = set(man_list) & set([x[0] for x in has_sibling_relation])
twin_sisters = set(woman_list) & set([x[0] for x in has_twin_relation])
twin_brothers = set(man_list) & set([x[0] for x in has_twin_relation])
sisters = set(woman_list) & set(x[1] for x in has_sibling_relation)

grandmothers = set(woman_list) & set([x[0] for x in has_grandparent_relation])
grandfathers = set(man_list) & set([x[0] for x in has_grandparent_relation])
granddaughters = set(woman_list) & set([x[0] for x in has_grandchild_relation])
grandsons = set(man_list) & set([x[0] for x in has_grandchild_relation])

# mothers, fathers, daughters, sons, grandmothers, grandfathers, granddaughters, grandsons, sisters, brothers,twin_sisters,twin_brothers

In [16]:
def inferences():
    if child_property != None:
        print("child done")
        create_inferrence(parent, child_property, child)

    if parent_property != None:
        print("parent done")
        create_inferrence(child2, parent_property, parent2)
    
    if grandchild_property != None:
        print("grand child done")
        create_inferrence(grandparent, grandchild_property, grandchild)

    if grandparent_property != None:
        print("grand parent done")
        create_inferrence(grandchild2, grandparent_property, grandparent2)
        
    if husband_property != None:
        print("husband done")
        create_inferrence(marriage3, husband_property, marriage4)

    if wife_property != None:
        print("wife done")
        create_inferrence(marriage, wife_property, marriage2)

    if ex_husband_property!= None:
        print("ex husband done")
        create_inferrence(marriage7, ex_husband_property, marriage8)

    if ex_wife_property!= None:
        print("ex wife done")
        create_inferrence(marriage5, ex_wife_property, marriage6)

    if cousin_property!= None:
        print("cousin done")
        create_inferrence(cousin1, cousin_property, cousin2)

    if partner_property!= None:
        print("partner done")
        create_inferrence(couple, partner_property, couple2)

    if father_property!= None:
        print("father done")
        create_gender_based_inferrence(has_child_relation, father_property, fathers)

    if mother_property!= None:
        print("mother done")
        create_gender_based_inferrence(has_child_relation, mother_property, mothers)

    if son_property!= None:
        print("son done")
        create_gender_based_inferrence(has_parent_relation, son_property, sons)

    if daughter_property!= None:
        print("daughter done")
        create_gender_based_inferrence(has_parent_relation, daughter_property, daughters)

    if grandson_property!= None:
        print("grandson done")
        create_gender_based_inferrence(has_grandchild_relation, grandson_property, grandsons)

    if granddaugther_property!= None:
        print("granddaughter done")
        create_gender_based_inferrence(has_grandchild_relation, granddaugther_property, granddaughters)

    if grandfather_property!= None:
        print("grandfather done")
        create_gender_based_inferrence(has_grandparent_relation, grandfather_property, grandfathers)

    if grandmother_property!= None:
        print("grandmother done")
        create_gender_based_inferrence(has_grandparent_relation, grandmother_property, grandmothers)

    if brother_property!= None:
        print("brother done")
        create_gender_based_inferrence(has_sibling_relation, brother_property, brothers)

    if sister_property != None:
        print("sister done")
        create_gender_based_inferrence(has_sibling_relation, sister_property, sisters)

    if twin_property != None:
        print("twin done")
        create_gender_based_inferrence(has_twin_relation, twin_property, twin_sisters)
        create_gender_based_inferrence(has_twin_relation, twin_property, twin_brothers)
        

    if sibling_property != None:
        print("sibling done")
        create_inferrence(sibling1, sibling_property, sibling2)  
        
    else:
        print("None")

In [17]:
inferences()

child done
parent done
husband done
wife done
ex husband done
ex wife done
cousin done
partner done
father done
mother done
son done
daughter done
grandmother done
brother done
sister done
twin done
sibling done


## Show dataframes of the inferences that are made. 

In [18]:
df2 = df2.drop_duplicates().sort_values(by=['Property'])
df2.reset_index(drop=True)

,Subject,Property,Object
0,Leonard_Green,has_Child,Rachel
1,Charles_Bing,has_Child,Chandler_Bing
2,Jack_Geller,has_Child,Monica
3,Jack_Geller,has_Child,Ross
4,Phoebe_Abbott,has_Child,Phoebe
5,Nora_Tyler_Bing,has_Child,Chandler_Bing
6,Judy_Geller,has_Child,Ross
7,Bitsy_Hannigan,has_Child,Mike_Hannigan
8,Theodore_Hannigan,has_Child,Mike_Hannigan
9,Gloria_Tribbiani,has_Child,Joey_Tribbiani


In [19]:
df5 = df5.drop_duplicates().sort_values(by=['Property'])
df5.reset_index(drop=True)

,Subject,Property,Object
0,Ben,has_Father,Ross
1,Chandler_Bing,has_Mother,Nora_Tyler_Bing
2,Jill_Green,has_Sister,Rachel
3,Jill_Green,has_Sister,Amy_Green
4,Dina,has_Sister,Mary_Angela_Tribbiani
5,Rachel,has_Sister,Amy_Green
6,Amy_Green,has_Sister,Rachel
7,Frank_Buffay,has_Sister,Phoebe
8,Nora_Tyler_Bing,has_Son,Chandler_Bing
9,Jack_Geller,has_Son,Ross


In [20]:
serialize()

@prefix : <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1> .
@prefix kinship1: <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

kinship1:Andrea_Waltham a kinship1:Human,
        kinship1:woman,
        owl:NamedIndividual ;
    kinship1:has_Daughter kinship1:Emily_Waltham .

kinship1:Brother_in_law a owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Brother in law"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    rdfs:subPropertyOf kinship1:In_law ;
    owl:propertyDisjointWith kinship1:Sister_in_law .

kinship1:Cousins_husband a owl:AsymmetricProperty,
        owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Cousins husband"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    owl:propertyDisjointWith kinship1:Cousins_wife .

kinship1:Daugther_in_law

# Add sibling relations

In [21]:
sibling_infer_relation_list = []
sibling_infer_relation_list_show = []
name_list_sibling = []

for s,p,o in g:
    if shorten(p) == "kinship1:has_Sibling" or relation(p) == "has_Sister" or relation(p) == "has_Brother"  or relation(p) == "has_twin":
        sibling_infer_relation_list_show.append((relation(s), relation(o)))
        sibling_infer_relation_list.append((s, o))
        name_list_sibling.append(relation(s))
        name_list_sibling.append(relation(o))

In [22]:
sibling_infer_relation_list_show

[('Tina', 'Joey_Tribbiani'),
 ('Dina', 'Joey_Tribbiani'),
 ('Joey_Tribbiani', 'Mary_Therese_Tribbiani'),
 ('Amy_Green', 'Rachel'),
 ('Joey_Tribbiani', 'Veronica'),
 ('Cookie', 'Joey_Tribbiani'),
 ('Joey_Tribbiani', 'Dina'),
 ('Mary_Angela_Tribbiani', 'Dina'),
 ('Jill_Green', 'Rachel'),
 ('Joey_Tribbiani', 'Mary_Angela_Tribbiani'),
 ('Dina', 'Mary_Angela_Tribbiani'),
 ('Monica', 'Ross'),
 ('Joey_Tribbiani', 'Tina'),
 ('Phoebe', 'Ursula_Buffay'),
 ('Amy_Green', 'Jill_Green'),
 ('Ross', 'Monica'),
 ('Mary_Angela_Tribbiani', 'Joey_Tribbiani'),
 ('Rachel', 'Jill_Green'),
 ('Amy_Green', 'Rachel'),
 ('Rachel', 'Amy_Green'),
 ('Gina', 'Joey_Tribbiani'),
 ('Jill_Green', 'Amy_Green'),
 ('Jill_Green', 'Amy_Green'),
 ('Joey_Tribbiani', 'Cookie'),
 ('Veronica', 'Joey_Tribbiani'),
 ('Mary_Therese_Tribbiani', 'Joey_Tribbiani'),
 ('Rachel', 'Amy_Green'),
 ('Dina', 'Mary_Angela_Tribbiani'),
 ('Phoebe', 'Frank_Buffay'),
 ('Frank_Buffay', 'Phoebe'),
 ('Frank_Buffay', 'Phoebe'),
 ('Jill_Green', 'Rachel'),

In [23]:
name_data = pd.DataFrame(name_list_sibling, columns = ["Names Of All Siblings"])
final_name_list = name_data.drop_duplicates().reset_index(drop=True)
final_name_list

,Names Of All Siblings
0,Tina
1,Joey_Tribbiani
2,Dina
3,Mary_Therese_Tribbiani
4,Amy_Green
5,Rachel
6,Veronica
7,Cookie
8,Mary_Angela_Tribbiani
9,Jill_Green


### Put all the individuals that are siblings of each other in a set together and put those sets in a list. 

In [24]:
synom = sibling_infer_relation_list
show = sibling_infer_relation_list_show
word_classes = make_word_classes(synom)
# print(word_classes)
print(make_word_classes(show))

[{'Tina', 'Dina', 'Mary_Angela_Tribbiani', 'Gina', 'Cookie', 'Veronica', 'Mary_Therese_Tribbiani', 'Joey_Tribbiani'}, {'Jill_Green', 'Amy_Green', 'Rachel'}, {'Ross', 'Monica'}, {'Ursula_Buffay', 'Frank_Buffay', 'Phoebe'}]


In [25]:
parent_infer_relation_list = []
parent_infer_relation_list_show = []
name_list_children = []
name_list_show = []

for s,p,o in g:
    if shorten(p) == "kinship1:has_parent" or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        parent_infer_relation_list_show.append((relation(s), relation(o)))
        parent_infer_relation_list.append((s, o))
        name_list_show.append(relation(s))
        name_list_children.append(s)
parent_infer_relation_list_show
name_list_children

[rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Rachel'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Joey_Tribbiani'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Monica'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Ben'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Joey_Tribbiani'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Ben'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Mike_Hannigan'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Rachel'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Ross'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Mike_Hannigan'),
 rdflib.term.URIRef('http://www.semanticweb.org/selin/onto

In [26]:
name_data = pd.DataFrame(name_list_show, columns = ["Names Of All children"])
final_name_list = name_data.drop_duplicates().reset_index(drop=True)
final_name_list

,Names Of All children
0,Rachel
1,Joey_Tribbiani
2,Monica
3,Ben
4,Mike_Hannigan
5,Ross
6,Chandler_Bing
7,Phoebe


### Create a list of the siblings and infer the sibling relations between them. 

In [27]:
new_list = []
end_list = []

for x in word_classes:
    for p in x:
        new_list.append(p)
        if p not in name_list_children:
            new_list.remove(p)
    
for x in word_classes:
    for people in new_list:
        for p in x:
            if p == people:
                end_list.append(x)     
end_list

[{rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Cookie'),
  rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Dina'),
  rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Gina'),
  rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Joey_Tribbiani'),
  rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Mary_Angela_Tribbiani'),
  rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Mary_Therese_Tribbiani'),
  rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Tina'),
  rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Veronica')},
 {rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Amy_Green'),
  rdflib.term.URIRef('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#Jill_Green'),
  rdflib.term.URIRef('http:

In [28]:
inferences()

child done
parent done
husband done
wife done
ex husband done
ex wife done
cousin done
partner done
father done
mother done
son done
daughter done
grandmother done
brother done
sister done
twin done
sibling done


In [29]:
df2 = df2.drop_duplicates().sort_values(by=['Property'])
df2.reset_index(drop=True)

,Subject,Property,Object
0,Leonard_Green,has_Child,Rachel
1,Sandra_Green,has_Child,Rachel
2,Joseph_Tribbiani,has_Child,Joey_Tribbiani
3,Ross,has_Child,Ben
4,Gloria_Tribbiani,has_Child,Joey_Tribbiani
5,Theodore_Hannigan,has_Child,Mike_Hannigan
6,Bitsy_Hannigan,has_Child,Mike_Hannigan
7,Judy_Geller,has_Child,Monica
8,Nora_Tyler_Bing,has_Child,Chandler_Bing
9,Phoebe_Abbott,has_Child,Phoebe


# Show final ontology

In [30]:
df5 = df5.drop_duplicates().sort_values(by=['Property'])
df5.reset_index(drop=True)

,Subject,Property,Object
0,Ben,has_Father,Ross
1,Chandler_Bing,has_Mother,Nora_Tyler_Bing
2,Jill_Green,has_Sister,Rachel
3,Jill_Green,has_Sister,Amy_Green
4,Dina,has_Sister,Mary_Angela_Tribbiani
5,Rachel,has_Sister,Amy_Green
6,Amy_Green,has_Sister,Rachel
7,Frank_Buffay,has_Sister,Phoebe
8,Charles_Bing,has_Son,Chandler_Bing
9,Bitsy_Hannigan,has_Son,Mike_Hannigan


In [31]:
serialize()

@prefix : <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1> .
@prefix kinship1: <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

kinship1:Andrea_Waltham a kinship1:Human,
        kinship1:woman,
        owl:NamedIndividual ;
    kinship1:has_Daughter kinship1:Emily_Waltham .

kinship1:Brother_in_law a owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Brother in law"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    rdfs:subPropertyOf kinship1:In_law ;
    owl:propertyDisjointWith kinship1:Sister_in_law .

kinship1:Cousins_husband a owl:AsymmetricProperty,
        owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Cousins husband"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    owl:propertyDisjointWith kinship1:Cousins_wife .

kinship1:Daugther_in_law

In [32]:
save('new_kinship2_ontology.ttl')

# Show the result of the counter of the newly added relations to the ontology


In [33]:
new_relations_counter

167